# Example Query for Survival Probability of 1-hop Queries

Queries our system in the form of:<br>
$P(survival\_time > X | Mut\_g_1 = True, Mut\_g_2 = True, ..., Mut\_g_n = True, Drug = d_1, Drug = d_2, ..., Drug = d_n)$<br>
Returned is a knowledge graph containing probability of survival time.

In [6]:
import requests
import json
import csv

In [7]:
# Function: buildQuery
#
# Input:
# -----------
# list of gene and drug tuples
#
# Output:
# -----------
# A query graph that answers this probablistic question: 
# P(survival_time > X | Mut_g1 = True, Mut_g2 = True, ..., Mut_gn = True, Drug = d1, Drug = d2, ..., Drug = dn)

def buildQuery(st, genes=[], drugs=[]):
    
    # empty response
    reasoner_std = { "query_graph": dict()}
    
    # empty query graph
    reasoner_std["query_graph"] = { "edges": [],
                                    "nodes": []
                                  }
    
    node_count = 0
    edge_count = 0
    
    # add genes
    for gene in genes:
        reasoner_std['query_graph']['nodes'].append({ 'id':'n{}'.format(node_count),
                                                      'type':'Gene',
                                                      'curie':'{}'.format(gene[1])
                                                   })
        node_count += 1
    
    # add drugs
    for drug in drugs:
        reasoner_std['query_graph']['nodes'].append({ 'id':'n{}'.format(node_count),
                                                      'type':'Drug',
                                                      'curie':'{}'.format(drug[1])
                                                   })
        node_count += 1
    
    # add in disease node
    disease = ('Breast_Cancer', 'MONDO:0007254')
    reasoner_std['query_graph']['nodes'].append({ 'id':'n{}'.format(node_count),
                                                  'type':'disease',
                                                  'curie':'{}'.format(disease[1])
                                                })
    node_count += 1
    
    # link all evidence to disease
    for node in reasoner_std['query_graph']['nodes']:
        if node['type'] == 'Gene':
            id = node['id']
            reasoner_std['query_graph']['edges'].append({ 'id':'e{}'.format(edge_count),
                                                          'type':'gene_to_disease_association',
                                                          'source_id':'{}'.format(id),
                                                          'target_id':'n{}'.format(node_count-1)
                                                       })
            edge_count += 1
        elif node['type'] == 'Drug':
            id = node['id']
            reasoner_std['query_graph']['edges'].append({ 'id':'e{}'.format(edge_count),
                                                          'type':'chemical_to_disease_or_phenotypic_feature_association',
                                                          'source_id':'{}'.format(id),
                                                          'target_id':'n{}'.format(node_count-1)
                                                       })
            edge_count += 1
            
    # add target survival node
    phenotype = ('Survival_Time', 'EFO:0000714')
    reasoner_std['query_graph']['nodes'].append({ 'id': 'n{}'.format(node_count),
                                                  'type': 'PhenotypicFeature',
                                                  'curie': '{}'.format(phenotype[1]),
                                                })
    node_count += 1
    
    # link disease to target
    reasoner_std['query_graph']['edges'].append({ 'id':'e{}'.format(edge_count),
                                                  'type':'disease_to_phenotype_association',
                                                  'value':970,
                                                  'source_id':'n{}'.format(node_count-2),
                                                  'target_id':'n{}'.format(node_count-1)
                                                })
    return reasoner_std
    

In [8]:
def readGenes():
    with open('gene_curie_map.csv', 'r') as gene_file:
        reader = csv.reader(gene_file)
        next(reader)
        rows = [row for row in reader]
    return rows

In [9]:
def readDrugs():
    with open('drug_curie_map.csv', 'r') as drug_file:
        reader = csv.reader(drug_file)
        next(reader)
        rows = [row for row in reader]
    return rows

In [10]:
# list of genes we can query over
#gene_list = readGenes()

# list of genes we can query over
#drug_list = readDrugs()

# or pull from above lists
genes = [('RAF1', 'ENSEMBL:ENSG00000132155'),
         ('BRCA1', 'ENSEMBL:ENSG00000012048')]
drugs = [('CYCLOPHOSPHAMIDE', 'CHEMBL:CHEMBL88')]
survival_time = 1000

# if genes or drugs is an empty list you can omit them
query = buildQuery(survival_time, genes=genes, drugs=drugs)
query['reasoner_id'] = 'exploring'
payload = {'query': query}
r = requests.post('http://chp.thayer.dartmouth.edu/submitQuery/', json=payload)
chp_res = json.loads(r.content)

QG = chp_res['query_graph']
KG = chp_res['knowledge_graph']
res = chp_res['results']

# extract probability
for edge in KG['edges']:
    if edge['type'] == 'disease_to_phenotype_association':
        p_survival = edge['has_confidence_level']

# probability of surival given QG specification
print("Probability of survival > {} days is:".format(survival_time), p_survival)

Probability of survival > 1000 days is: 0.4
